# The Julia Programming Language

## Multiple Dispatch
Multiple dispatch is a fundamental paradigm of the Julia programming language. It is the selection of a function implementation based on the (_dynamic_) types of each argument of the function. It is not only a nice notation to remove a long list of “case” statements, but it is part of the reason for Julia’s speed.
You can read more about multiple dispatch in general, and its implementation in Julia here:
 - Julia: A fresh approach to numerical computing ([arXiv paper](https://arxiv.org/pdf/1411.1607.pdf))
 - [Wikipedia](https://en.wikipedia.org/wiki/Multiple_dispatch)
 - The Julia documentation on [Methods](https://docs.julialang.org/en/v1/manual/methods/)
 - [Video "The Unreasonable Effectiveness of Multiple Dispatch" by Stefan Karpinski](https://www.youtube.com/watch?v=kc9HwsxE1OY)

Other flavours of dispatch are:
- Static single dispatch: not implemented in practice.
- Static multiple dispatch: this implementation is common in statically typed languages (e.g., overloading in C++).
- Dynamic single dispatch: in languages like MATLAB and Python.
- Dynamic multiple dispatch: this is the implementation in Julia, typically referred to as "multiple dispatch".

### Example 1: 

### Example 2:

## Lazy Evaluation
Lazy evaluation is the cornerstone of many functional programming languages. The key concept is as follows: when an operation such as a functional call is performed, we do not evaluate the result immediately (that would be _eager_ evaluation). Instead, the operation is postponed until the result is actually necessary, i.e., it is evaluated _lazily_.

Laziness is exploited in several packages in Julia: ``Lazy.jl``, ``LazyArrays.jl``, and many others (see https://www.juliapackages.com/packages?search=lazy). The finite element method package ``Gridap.jl`` also makes use of laziness to achieve very high performance.

### Laziness in Julia
Even without using any additional packages, laziness is incorporated into several aspects of Julia:
 - Generators and ranges
 - 

### Example: Ranges
When using ranges or generators, the result will not be evaluated immediately. Instead, lazy evaluation is used to reduce computation time and storage until (part of) the result is necessary. This is illustrated using a range of integers in this example.

In [3]:
using BenchmarkTools

In [5]:
@btime 1:10000000
@btime collect(1:10000000)

  0.001 ns (0 allocations: 0 bytes)
  13.264 ms (2 allocations: 76.29 MiB)


10000000-element Vector{Int64}:
        1
        2
        3
        4
        5
        6
        7
        8
        9
       10
       11
       12
       13
        ⋮
  9999989
  9999990
  9999991
  9999992
  9999993
  9999994
  9999995
  9999996
  9999997
  9999998
  9999999
 10000000

### ``LazyArrays.jl``
Lazy arrays allow for the lazy evaluation of operations on arrays and matrices. This includes specific algebraic operations, matrix solve, kronecker product, broadcasting, etc.
The big advantage is that operations can be _compounded_ by not evaluating them immediately, but keeping track of which operations are to be performed on the array. 
- This allows for optimizations to be applied for certain compound operations that can be much faster than if every operation was evaluated eagerly.
- Additionally, memory usage can often be kept low when using lazy arrays: there is no need to allocate any memory for the result because it is not computed immediately. Only when the result is actually needed, it will be computed on-demand.

In [2]:
using LazyArrays

### Example 1: Optimized Matrix Operations

In [3]:
N = 5;
A = randn(N,N); x = randn(N); y = randn(N); z1 = similar(y); z2 = similar(y);

function fun(A, x, y, z)
    z .= @~ 2.0 * A * x + 3.0 * y
end
z1 = @btime fun(A, x, y, z1)       # Optimized
z2 = @btime 2.0 * A * x + 3.0 * y  # Not optimized

z1 ≈ z2      # But the result is the same

  95.158 ns (0 allocations: 0 bytes)
  207.719 ns (3 allocations: 288 bytes)


true

### Laziness in ``Gridap.jl``
Laziness is used extensively in ``Gridap.jl``, and is the one of the reasons for its excellent performance. Lazy arrays play a fundamental role in the implementation of the finite element method in Gridap (how it works is explained in https://gridap.github.io/Tutorials/dev/pages/t013_poisson_dev_fe/).
A large amount of code is devoted to lazy operations in Gridap, mostly centered around the ``lazy_map`` function. It creates a ``LazyArray`` that conceptually applies the mapping function to an array, but does not actually execute this until it is necessary. Large and complex 'compound' operations are built in this way. Eventually, the mapping may be evaluated at a set of points using evaluate.

Lazy objects constructed in this way provide the following advantages in Gridap:
 - Memory allocation (and consumption) is kept at very low levels, since ``lazy_map`` will never return an array that stores the result at all cells at once.
 - Computation can be done in an efficient way (e.g., using cache to store the entry-wise data without the need to allocate memory each time we access the ``LazyArray``).
 - The recursive application of ``lazy_map`` lets us build complex operation trees among arrays of ``Maps`` as the ones required for the implementation of variational forms. While building these trees, by virtue of Julia support for multiple type dispatching, there are plenty of opportunities for optimization by changing the order in which the operations are performed.
 - Using ``lazy_map`` we are hiding thousands of cell loops across the code. As a result, Gridap is much more expressive for cell-wise implementations.
 - ``lazy_map`` allows for very expressive code that reads much like the original mathematical formulation both internally (e.g., in pretty much every aspect of low-level Gridap code) and externally (e.g., the definition of the variational form, shown below).

In [ ]:
res(u, v) = ∫( ∇(u) ⋅ ∇(v) )dΩ - ∫( fsource * v  )dΩ;

Compare this to the original mathematical expression:
$$ \int \nabla u \cdot \nabla v d\Omega = \int f v d\Omega $$

The computational back-end of Gridap is discussed in the following paper: https://www.researchgate.net/publication/354890339_The_software_design_of_Gridap_a_Finite_Element_package_based_on_the_Julia_JIT_compiler

## Callable Structs

## Automatic Differentiation
In general, automatic differentiation (AD) is a set of techniques to evaluate the derivative of a function specified by a computer program. By applying the chain rule repeatedly to these operations, derivatives of arbitrary order can be computed automatically, accurately to working precision, and using at most a small constant factor more arithmetic operations than the original program [[Wikipedia](https://en.wikipedia.org/wiki/Automatic_differentiation)].

Several Julia packages have been written that are related to automatic differentiation:
 - Zygote.jl
 - Enzyme.jl
 - ChainRules.jl

### Example ``Zygote.jl``

### Example ``ChainRules.jl``

## Map, Filter, and Reduce
The functions `map`, `filter`, and `reduce` are not exclusive to Julia, but are an important part of any functional programming language. They are so-called _higher-order functions_, which means that they can take other functions as arguments. Using these functions, very expressive code can be written (which typically means for-loops can be avoided).

Note that these functions perform very well in conjuction with the lazy evaluation concept discussed above.

### Map
`map` applies a function `f` to every element of a collection `c` and returns a collection containing the results. The syntax is
```julia
map(f, c...)        # Apply 'f' to 'c' and return a new collection
map!(f, dest, c)    # Apply 'f' to 'c' and store the result in 'dest'
```
For example, if we want to multiply all odd numbers in an array by $3$, this can be easily achieved using a `map`.

In [4]:
a = collect(1:20);                      # Input collection of numbers
b = map(a) do elem                      # Perform a map on the collection a
    isodd(elem) ? 3 * elem : elem;      #    for each element: return 3 * elem  if it is odd, 
end;                                    #                      or         elem  if it is even

print(a, "\n")
print(b, "\n")

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
[3, 2, 9, 4, 15, 6, 21, 8, 27, 10, 33, 12, 39, 14, 45, 16, 51, 18, 57, 20]


### Filter
`filter` returns a copy of a collection `a`, removing the elements for which a function `f` is false.
```julia
filter(f, a)    # Return a copy of 'a' containing only the elements for which 'f' is true
```

In [5]:
a = collect(1:10);        # Input collection of numbers
b = filter(isodd, a);     # Return only those elements which are odd

print(a, "\n")
print(b, "\n")

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[1, 3, 5, 7, 9]


### Reduce
`reduce` takes a collection of elements `itr` and reduces them to a single element, following the rule given by the operator `op`. The optional value `init` is returned for empty collections. It is often used to sum/multiply/subtract/divide, but could also be used to concatenate lists.
```julia
reduce(op, itr; [init])    # Reduce collection 'itr' using operator 'op' and return a new collection
```
The reduce operation is also known as `fold`. If associativity of the reduction must be guaranteed, the functions `foldl` and `foldr` can be used (for left or right associativity, respectively).

In [6]:
a = collect(1:10);        # Input collection of numbers
b = reduce(+, a);         # Reduce the collection by adding all elements together

print(a, "\n")
print(b, "\n")

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
55


The `map` and `reduce` actions can be combined in a single call to `mapreduce(f, op, itrs...)`, where `f` is applied to each element in `itrs` and the result is reduced using the binary function `op`.

In [7]:
# TODO mapreduce() example